In [26]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.merrill import Merrill
from transformer.date_transformer import DateTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from extractor.finnhub_extractor import FinnhubExtractor
from extractor.tiingo_extractor import TiingoExtractor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import pandas_datareader as pdr
from time import sleep
from trader.IBApp import IBApp
import asyncio
import numpy as np

In [27]:
## Loading Constants
end = datetime.now().strftime("%Y-%m-%d")
start = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("portfolio")
back_db = Strategy("unity")
market = Market()
merrill = Merrill()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = True
year = datetime.now().year
quarter = math.ceil((datetime.now().month - 1) // 3) + 1
datasets = ["pdr"
#             ,"tiingo"
#             ,"finnhub"
           ]
delta_req = 1.05
daily_delta_req = float(np.log(1+delta_req)/365)
epoch = 77
dr = 1.05
bond = False
bond_value = False
bond_classification = False
ceiling = True
weekly = True
weekly_value = False
weekly_classification = False
fundamental = True
fundamental_value = False
fundamental_classification = False

In [25]:
recs = []
fails = []
price_dataframes = []
strat_db.connect()
dataset = "pdr"
fundamental_prices = strat_db.retrieve_data("{}_fundamental_quarterly_sim".format(dataset)).drop("_id",axis=1)
bond_prices = strat_db.retrieve_data("{}_bond_weekly_sim".format(dataset)).drop("_id",axis=1)
weekly_prices = strat_db.retrieve_data("{}_weekly_recs".format(dataset)).drop("_id",axis=1)
for ticker in tqdm(tickers["Symbol"]):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        print(ticker)
        fp = FinnhubExtractor.quote(ticker)
        price = pd.DataFrame([fp])
        sleep(1)
        price["ticker"] = ticker
        if dataset == "pdr":
            price = ColumnTransformer.rename_columns(price, " ")
        else:
            price = ColumnTransformer.rename_columns(price, "_")
        price_dataframes.append(price)
    except Exception as e:
        msg = {"dataset":dataset,"ticker":ticker,"exception":str(e)}
        print(msg)
        fails.append(ticker)
        continue
prices = pd.concat(price_dataframes)
prices["date"] = datetime.now()
prices["adjclose"]  = prices["c"]
test = prices[["date","ticker","adjclose"]].merge(weekly_prices,on="ticker",how="left").merge(fundamental_prices,on="ticker",how="left")
test = test.merge(bond_prices,on="ticker",how="left")
test["fundamental_predicted_delta"] = (test["quarterly_fundamental_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["weekly_predicted_delta"] = (test["weekly_price_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["bond_predicted_delta"] = (test["weekly_bond_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["dataset"] = dataset
recs.append(test)
r = pd.concat(recs).sort_values("weekly_predicted_delta",ascending=False).dropna()
r["calculation_date"] = datetime.now()
strat_db.connect()
strat_db.store_data("recommendations",r)
strat_db.close()

  0%|                                                                                                                                                                                            | 0/506 [00:00<?, ?it/s]

A


  0%|▎                                                                                                                                                                                 | 1/506 [00:08<1:07:55,  8.07s/it]

AAL


  0%|▋                                                                                                                                                                                   | 2/506 [00:09<50:24,  6.00s/it]

...

AZO


 11%|███████████████████▍                                                                                                                                                               | 55/506 [01:16<09:32,  1.27s/it]

BA


 11%|███████████████████▍                                                                                                                                                               | 55/506 [01:17<10:33,  1.41s/it]


KeyboardInterrupt: 

In [ ]:
test = prices[["date","ticker","adjclose"]].merge(weekly_prices,on="ticker",how="left").merge(fundamental_prices,on="ticker",how="left")
test = test.merge(bond_prices,on="ticker",how="left")
test["fundamental_predicted_delta"] = (test["quarterly_fundamental_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["weekly_predicted_delta"] = (test["weekly_price_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["bond_predicted_delta"] = (test["weekly_bond_regression_prediction"] - test["adjclose"]) / test["adjclose"]
test["dataset"] = dataset
r = test.sort_values("weekly_predicted_delta",ascending=False)
r["calculation_date"] = datetime.now()

In [ ]:
sim = r

In [ ]:
sim

In [ ]:
daily_delta_req = float(np.log(1+delta_req)/365)
if weekly:
    delta_column = "weekly_predicted_delta"
    hpr = 7
if fundamental and not weekly:
    delta_column = "fundamental_predicted_delta"
    hpr = 90
if bond and not weekly and not fundamental:
    delta_column = "bond_predicted_delta"
    hpr = 7
if not weekly_value:
    sim["weekly_predicted_delta"] = sim["weekly_predicted_delta"] * -1
    sim["weekly_price_classification_prediction"] = [not x for x in sim["weekly_price_classification_prediction"]]
if not bond_value:
    sim["bond_predicted_delta"] = sim["bond_predicted_delta"] * -1
    sim["weekly_bond_classification_prediction"] = [not x for x in sim["weekly_bond_classification_prediction"]]
if not fundamental_value:
    sim["fundamental_predicted_delta"] = sim["fundamental_predicted_delta"] * -1
    sim["quarterly_fundamental_classification_prediction"] = [not x for x in sim["quarterly_fundamental_classification_prediction"]]
if ceiling:
    for col in sim.columns:
            if 'delta' in col:
                sim["{}_abs".format(col)] = [abs(x) for x in sim[col]]
                filtered_sim = sim[(sim["{}_abs".format(col)] <= 1)]
else:
    filtered_sim = sim
filtered_sim = filtered_sim[(filtered_sim[delta_column] >= float((1+daily_delta_req)**hpr) - 1)]
if fundamental_classification:
    filtered_sim = filtered_sim[filtered_sim["quarterly_fundamental_classification_prediction"] == 1]
    filtered_sim = filtered_sim[filtered_sim["quarterly_fundamental_classification_score"] >= 0.7]
if weekly_classification:
    filtered_sim = filtered_sim[filtered_sim["weekly_price_classification_prediction"] == 1]
    filtered_sim = filtered_sim[filtered_sim["weekly_price_classification_score"] >= 0.7]
if bond_classification:
    filtered_sim = filtered_sim[filtered_sim["weekly_bond_classification_prediction"] == 1]
    filtered_sim = filtered_sim[filtered_sim["weekly_bond_classification_score"] >= 0.7]
if fundamental:
    filtered_sim = filtered_sim[filtered_sim["fundamental_predicted_delta"] >= float((1+daily_delta_req)**90 - 1)]
    filtered_sim = filtered_sim[(filtered_sim["quarterly_fundamental_regression_score"] > 0) & (filtered_sim["quarterly_fundamental_regression_score"] < 0.1)]
if weekly:
    filtered_sim = filtered_sim[filtered_sim["weekly_predicted_delta"] >= float((1+daily_delta_req)**7 - 1)]
    filtered_sim = filtered_sim[(filtered_sim["weekly_price_regression_score"] > 0) & (filtered_sim["weekly_price_regression_score"] < 0.1)]
if bond:
    filtered_sim = filtered_sim[filtered_sim["bond_predicted_delta"] >= float((1+daily_delta_req)**7 - 1)]
    filtered_sim = filtered_sim[(filtered_sim["weekly_bond_regression_score"] > 0) & (filtered_sim["weekly_bond_regression_score"] < 0.1)]

In [ ]:
filtered_sim.sort_values("weekly_predicted_delta",ascending=False)

In [ ]:
blacklist = ["BRK-B","AMZN","GOOGL","MSFT","AAPL","NFLX","PRGO"]
if reload:
    for col in r.columns:
        if "delta" in col:
            r = r[(r[col]>=-1) & (r[col]<=1)]
        else:
            continue
    available_seats = 9
    offerings = filtered_sim.sort_values("weekly_predicted_delta",ascending=False).head(available_seats)
    trades = []
    portfolio_pv = 180000
    current_hpr = 7
    for i in range(len(offerings)):
        ticker_rec = offerings.iloc[i]
        ticker_limit = ticker_rec["adjclose"]
        gainz = float((1+daily_delta_req)**current_hpr)
        ticker_sell_limit = round(ticker_limit * gainz,2)
        quantity = round(portfolio_pv / available_seats / ticker_limit)
        trades.append({"ticker":ticker_rec["ticker"]
                       ,"quantity":quantity
                       ,"weekly_predicted_delta":ticker_rec["weekly_predicted_delta"]
                       ,"fundamental_predicted_delta":ticker_rec["fundamental_predicted_delta"]
                       ,"buy_price":ticker_limit
                       ,"sell_price":ticker_sell_limit,"buy_date":datetime.now(),"calculation_date":datetime.now()})
    orders = pd.DataFrame(trades)
    strat_db.connect()
    strat_db.store_data("orders",orders)
    strat_db.close()
else:
    strat_db.connect()
    orders = strat_db.retrieve_data("orders")
    strat_db.close()

In [35]:
strat_db.connect()
orders = strat_db.retrieve_data("orders")
strat_db.close()

In [36]:
orders

,_id,ticker,quantity,weekly_delta,buy_price,sell_price,buy_date,calculation_date,weekly_predicted_delta,fundamental_predicted_delta
0,6021359860be2acc951132fe,VZ,966.0,0.063665,55.32,55.75,2021-02-08 07:58:42.206,2021-02-08 07:58:42.206,NaN,NaN
1,6021359860be2acc951132ff,ATO,600.0,0.057075,89.05,89.74,2021-02-08 07:58:42.206,2021-02-08 07:58:42.206,NaN,NaN
2,6021359860be2acc95113300,CNP,2498.0,0.033336,21.40,21.57,2021-02-08 07:58:42.207,2021-02-08 07:58:42.207,NaN,NaN
3,6021359860be2acc95113301,ED,748.0,0.025170,71.50,72.06,2021-02-08 07:58:42.208,2021-02-08 07:58:42.208,NaN,NaN
4,6021359860be2acc95113302,NI,2344.0,-0.009964,22.80,22.98,2021-02-08 07:58:42.208,2021-02-08 07:58:42.208,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
117,607d2db08e1979b7b9502bb5,PKI,150.0,NaN,133.16,135.00,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.092441,0.328746
118,607d2db08e1979b7b9502bb6,CLX,105.0,NaN,190.73,193.37,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.052889,0.239654
119,607d2db08e1979b7b9502bb7,CL,248.0,NaN,80.70,81.82,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.045781,0.257945
120,607d2db08e1979b7b9502bb8,DVA,177.0,NaN,113.11,114.68,2021-04-19 03:13:52.778,2021-04-19 03:13:52.778,0.026312,0.418072


In [37]:
orders.merge(tickers.rename(columns={"Symbol":"ticker"})[["ticker","GICS Sector"]],on="ticker",how="left").sort_values("weekly_predicted_delta",ascending=False)

,_id,ticker,quantity,weekly_delta,buy_price,sell_price,buy_date,calculation_date,weekly_predicted_delta,fundamental_predicted_delta,GICS Sector
107,607d2da08e1979b7b9502baa,LNT,534.0,NaN,56.160000,57.16,2021-04-19 03:13:36.904,2021-04-19 03:13:36.904,3635.498676,0.294627,Utilities
113,607d2db08e1979b7b9502bb1,LNT,356.0,NaN,56.160000,56.94,2021-04-19 03:13:52.776,2021-04-19 03:13:52.776,3635.498676,0.294627,Utilities
108,607d2da08e1979b7b9502bab,LLY,159.0,NaN,189.250000,192.63,2021-04-19 03:13:36.904,2021-04-19 03:13:36.904,3486.665125,0.382252,Health Care
114,607d2db08e1979b7b9502bb2,LLY,106.0,NaN,189.250000,191.87,2021-04-19 03:13:52.776,2021-04-19 03:13:52.776,3486.665125,0.382252,Health Care
115,607d2db08e1979b7b9502bb3,WEC,206.0,NaN,96.960000,98.30,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,2385.106384,0.210696,Utilities
...,...,...,...,...,...,...,...,...,...,...,...
87,606b280d14a9c5732c02e1ff,F,4371.0,0.383249,12.170000,12.34,2021-04-05 11:09:01.943,2021-04-05 11:09:01.943,NaN,NaN,Consumer Discretionary
88,606b280d14a9c5732c02e200,KIM,2770.0,0.379627,19.200001,19.47,2021-04-05 11:09:01.943,2021-04-05 11:09:01.943,NaN,NaN,Real Estate
89,606b280d14a9c5732c02e201,RF,2526.0,0.379292,21.059999,21.35,2021-04-05 11:09:01.943,2021-04-05 11:09:01.943,NaN,NaN,Financials
90,606b280d14a9c5732c02e202,RJF,420.0,0.377192,126.510002,128.26,2021-04-05 11:09:01.943,2021-04-05 11:09:01.943,NaN,NaN,Financials


In [40]:
orders = orders .sort_values("buy_date",ascending=False).head(9)

In [41]:
orders

,_id,ticker,quantity,weekly_delta,buy_price,sell_price,buy_date,calculation_date,weekly_predicted_delta,fundamental_predicted_delta
121,607d2db08e1979b7b9502bb9,TMO,41.0,NaN,492.92,499.75,2021-04-19 03:13:52.778,2021-04-19 03:13:52.778,0.024204,0.417336
120,607d2db08e1979b7b9502bb8,DVA,177.0,NaN,113.11,114.68,2021-04-19 03:13:52.778,2021-04-19 03:13:52.778,0.026312,0.418072
119,607d2db08e1979b7b9502bb7,CL,248.0,NaN,80.70,81.82,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.045781,0.257945
118,607d2db08e1979b7b9502bb6,CLX,105.0,NaN,190.73,193.37,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.052889,0.239654
117,607d2db08e1979b7b9502bb5,PKI,150.0,NaN,133.16,135.00,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.092441,0.328746
116,607d2db08e1979b7b9502bb4,QCOM,145.0,NaN,138.21,140.12,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,0.118390,0.465898
115,607d2db08e1979b7b9502bb3,WEC,206.0,NaN,96.96,98.30,2021-04-19 03:13:52.777,2021-04-19 03:13:52.777,2385.106384,0.210696
114,607d2db08e1979b7b9502bb2,LLY,106.0,NaN,189.25,191.87,2021-04-19 03:13:52.776,2021-04-19 03:13:52.776,3486.665125,0.382252
113,607d2db08e1979b7b9502bb1,LNT,356.0,NaN,56.16,56.94,2021-04-19 03:13:52.776,2021-04-19 03:13:52.776,3635.498676,0.294627


In [ ]:
app = IBApp()
app.connect("localhost",7491,777)
oid = 7777
for ticker in orders["ticker"]:
    order = orders[orders["ticker"] == ticker]
    app.execute_order(oid,order["ticker"].item(),"BUY",order["quantity"].item(),order["buy_price"].item())
    oid +=1
app.run()
app.disconnect()

In [42]:
app.connect("localhost",7491,777)
oid = 8000
for ticker in orders["ticker"]:
    order = orders[orders["ticker"] == ticker]
    app.execute_order(oid,order["ticker"].item(),"SELL",order["quantity"].item(),order["sell_price"].item())
    oid +=1
app.run()
app.disconnect()

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR:ibapi.wrapper:ERROR 8000 354 You are trying to submit an order without having market data for this instrument.
IB strongly recommends against this kind of blind trading
which may result in erroneous or unexpected trades.
Restriction is specified in Precautionary Settings of Global Configuration/Presets.
ERROR:ibapi.wrapper:ERROR 8001 354 You are trying to submit an order without having market data for this instrument.
IB strongly recommends against this kind of